In [ ]:
# imports

import os
import asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# Initialize OpenAI client
openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [ ]:
# Run the below command to install playwright chromium browser.  This must be run from command line in terminal
#uv run playwright install chromium


In [ ]:
# ============================================================================
# HOW TO EXTRACT COOKIES AND HEADERS FROM A REAL BROWSER
# ============================================================================

# METHOD 1: Extract Cookies using JavaScript in Chrome Console
# ----------------------------------------------------------------------------
# 1. Open the website in Chrome (make sure you can access it)
# 2. Open DevTools: Press F12 or Cmd+Option+I (Mac) / Ctrl+Shift+I (Windows)
# 3. Go to the Console tab
# 4. Paste and run this JavaScript code:
#
#    // Copy and paste this into Chrome Console to get all cookies:
#    (() => {
#        const hostname = window.location.hostname;
#        const baseDomain = hostname.startsWith('www.') ? hostname.substring(4) : hostname;
#        const cookies = document.cookie.split(';').map(c => {
#            const [name, ...rest] = c.trim().split('=');
#            return {
#                name: name.trim(), 
#                value: rest.join('='), 
#                domain: '.' + baseDomain,
#                path: '/'
#            };
#        }).filter(c => c.name && c.value);
#        console.log('// Copy this output:');
#        cookies.forEach(c => console.log(JSON.stringify(c) + ','));
#        return cookies;
#    })();
#
# 5. Copy all the output and paste it into the cookies list below

# METHOD 2: Extract Cookies from Application Tab
# ----------------------------------------------------------------------------
# 1. Open DevTools > Application tab (or Storage in older Chrome)
# 2. In left sidebar: Cookies > [your website URL]
# 3. Manually copy each cookie's name, value, domain, and path

# METHOD 3: Extract Headers from Network Tab
# ----------------------------------------------------------------------------
# 1. Open DevTools > Network tab
# 2. Reload the page (Cmd+R or F5)
# 3. Click on the first/main request (usually the HTML document)
# 4. Look at "Request Headers" section
# 5. Copy important headers like:
#    - Cookie (if you want to use Cookie header instead of cookies list)
#    - Referer
#    - Origin
#    - Authorization (if present)
#    - User-Agent (usually not needed, we set it automatically)

# ============================================================================
# PASTE YOUR COOKIES AND HEADERS HERE
# ============================================================================

# Replace these with your actual cookies from the browser
my_cookies = [
    # Example format:
    # {"name": "session_id", "value": "abc123xyz", "domain": ".bizbuysell.com", "path": "/"},
    # {"name": "csrf_token", "value": "def456", "domain": ".bizbuysell.com", "path": "/"},
    # Add more cookies here...
]

# Replace these with your actual headers (optional, but can help)
my_headers = {
    # 'Referer': 'https://www.bizbuysell.com/',
    'Referer': 'https://www.bizbuysell.com/georgia/restaurants-and-food-businesses-for-sale/?q=bHQ9MzAsNDAsODA%3D',
    # 'Origin': 'https://www.bizbuysell.com',
    'Origin': 'https://www.bizbuysell.com',
    # 'Cookie': 'session_id=abc123; csrf_token=def456',  # Alternative to cookies list
}

print("✅ Cookie and header extraction helpers loaded!")
print(f"\n📋 You have {len(my_cookies)} cookies and {len(my_headers)} headers configured")
print("\n💡 To use them, call scrape_website() like this:")
print("   html = await scrape_website(url, cookies=my_cookies, headers=my_headers)")


In [ ]:
# this function is used to scrape a website using Playwright
# Includes retry logic and better error handling for HTTP2 protocol errors
# Tries multiple browsers and configurations to handle HTTP2 errors

async def scrape_website(url, max_retries=3, timeout=30000, use_firefox_fallback=True, cookies=None, headers=None):
    """
    Scrape a website using Playwright with retry logic and HTTP2 error handling.
    
    Args:
        url: The URL to scrape
        max_retries: Maximum number of retry attempts per browser (default: 3)
        timeout: Page load timeout in milliseconds (default: 30000)
        use_firefox_fallback: Try Firefox if Chromium fails (default: True)
        cookies: List of cookie dicts with keys: name, value, domain, path (optional)
                Example: [{"name": "session", "value": "abc123", "domain": ".example.com", "path": "/"}]
        headers: Dict of HTTP headers to add (optional)
                Example: {"Referer": "https://example.com", "Cookie": "session=abc123"}
    
    Returns:
        HTML content as string
    """
    last_error = None
    
    # Try different browser configurations
    browser_configs = [
        {
            'name': 'Chromium (non-headless)',
            'browser_type': 'chromium',
            'headless': False,  # Some sites work better in non-headless
            'args': [
                '--disable-http2',
                '--disable-blink-features=AutomationControlled',
                '--disable-dev-shm-usage',
                '--no-sandbox'
            ]
        },
        {
            'name': 'Chromium (HTTP/2 disabled)',
            'browser_type': 'chromium',
            'headless': True,
            'args': [
                '--disable-http2',  # KEY FIX: Disable HTTP/2 to avoid protocol errors
                '--disable-blink-features=AutomationControlled',
                '--disable-dev-shm-usage',
                '--no-sandbox',
                '--disable-setuid-sandbox',
                '--disable-web-security',
                '--disable-features=IsolateOrigins,site-per-process'
            ]
        }
    ]
    
    # Add Firefox as fallback if enabled
    if use_firefox_fallback:
        browser_configs.append({
            'name': 'Firefox',
            'browser_type': 'firefox',
            'headless': True,
            'args': []
        })
    
    for config in browser_configs:
        print(f"Trying {config['name']}...")
        
        for attempt in range(max_retries):
            try:
                async with async_playwright() as p:
                    # Get the browser type
                    if config['browser_type'] == 'chromium':
                        browser = await p.chromium.launch(
                            headless=config['headless'],
                            args=config['args']
                        )
                    elif config['browser_type'] == 'firefox':
                        browser = await p.firefox.launch(
                            headless=config['headless'],
                            args=config['args']
                        )
                    else:
                        continue
                    
                    # Prepare headers - merge default headers with user-provided headers
                    default_headers = {
                        'Accept-Language': 'en-US,en;q=0.9',
                        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
                        'Accept-Encoding': 'gzip, deflate, br',
                        'Connection': 'keep-alive',
                        'Upgrade-Insecure-Requests': '1'
                    }
                    
                    # Merge with user-provided headers (user headers take precedence)
                    if headers:
                        default_headers.update(headers)
                    
                    # Create context with realistic browser settings
                    context_options = {
                        'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
                        'viewport': {'width': 1920, 'height': 1080},
                        'extra_http_headers': default_headers
                    }
                    
                    # Add cookies if provided
                    if cookies:
                        context_options['storage_state'] = {
                            'cookies': cookies,
                            'origins': []
                        }
                        print(f"📋 Using {len(cookies)} cookies from real browser session")
                    
                    context = await browser.new_context(**context_options)
                    
                    page = await context.new_page()
                    
                    # Add script to hide webdriver property
                    await page.add_init_script("""
                        Object.defineProperty(navigator, 'webdriver', {
                            get: () => undefined
                        });
                    """)
                    
                    # Try to navigate with different wait strategies
                    wait_strategies = ['domcontentloaded', 'load']
                    
                    for strategy in wait_strategies:
                        try:
                            await page.goto(url, wait_until=strategy, timeout=timeout)
                            break
                        except Exception as goto_error:
                            if strategy == wait_strategies[-1]:  # Last strategy
                                raise
                            continue  # Try next strategy
                    
                    # Wait a bit for any dynamic content to load
                    await page.wait_for_timeout(2000)
                    
                    # Get the content
                    content = await page.content()
                    
                    await browser.close()
                    print(f"✅ Successfully scraped with {config['name']}!")
                    return content
                    
            except Exception as e:
                last_error = e
                error_str = str(e)
                
                # Check if it's an HTTP2 error
                is_http2_error = 'ERR_HTTP2_PROTOCOL_ERROR' in error_str or 'HTTP2' in error_str.upper()
                
                if attempt < max_retries - 1:
                    wait_time = (attempt + 1) * 2
                    if is_http2_error:
                        print(f"⚠️  HTTP2 error on attempt {attempt + 1}. Retrying in {wait_time} seconds...")
                    else:
                        print(f"⚠️  Attempt {attempt + 1} failed: {error_str[:100]}... Retrying in {wait_time} seconds...")
                    await asyncio.sleep(wait_time)
                else:
                    print(f"❌ {config['name']} failed after {max_retries} attempts: {error_str[:150]}")
                    # Try next browser config
                    break
        
        # If we got here and have content, return it (shouldn't happen, but safety check)
        if 'content' in locals():
            return content
    
    # All configurations failed
    raise Exception(f"Failed to scrape {url} after trying all browser configurations. Last error: {str(last_error)}")

In [ ]:
# test website scraping
# If you're getting "Access Denied", uncomment the cookies and headers below

url = "https://www.bizbuysell.com/georgia/restaurants-and-food-businesses-for-sale"

# Option 1: Without cookies/headers (may work for some sites)
html_content = await scrape_website(url)

# Option 2: With cookies and headers from real browser (use if you get access denied)
# html_content = await scrape_website(url, cookies=my_cookies, headers=my_headers)

print(f"✅ Scraped {len(html_content)} characters of HTML")

In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a business buying assistant that analyzes the contents of a website,
and provides a list of the businesses for sale that includes business name, location, price, revenue, cashflow and best method to evaluate price of the business.
Ignore text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [ ]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a summary for every business listed.

"""

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

async def summarize(url):
    html_content = await scrape_website(url, cookies=my_cookies, headers=my_headers)
    #Get only the text parts of the webpage
    soup = BeautifulSoup(html_content, 'html.parser')
    html_text = soup.get_text(separator=' ', strip=True)
    response = openai.chat.completions.create(
        model = "gpt-4.1-nano",
        messages = messages_for(html_text)
    )
    return response.choices[0].message.content

In [ ]:
# A function to display this nicely in the output, using markdown

async def display_summary(url):
    summary = await summarize(url)
    display(Markdown(summary))

In [ ]:
await display_summary("https://www.bizbuysell.com/georgia/restaurants-and-food-businesses-for-sale/?q=bHQ9MzAsNDAsODA%3D")